# Generative Model Demo: Constructing a simple likelihood model 
This demo notebook provides a walk-through of how to build a simple A matrix (or likelihood mapping) that encodes an aegnt's beliefs about how hidden states 'cause' or probabilistically relate to observations

### Imports

First, import `pymdp` and the modules we'll need.

In [ ]:
import os
import sys
import pathlib

import numpy as np
import itertools
import pandas as pd

path = pathlib.Path(os.getcwd())
module_path = str(path.parent) + '/'
sys.path.append(module_path)

import pymdp.utils as utils
from pymdp.utils import create_A_matrix_stub, read_A_matrix
from pymdp.algos import run_vanilla_fpi

## The world (as represented by the agent's generative model)

### Hidden states

We assume the agent's "represents" (this should make you think: generative _model_ , not _process_ ) its environment using two latent variables that are statistically independent of one another - we can thus represent them using two _hidden state factors._

We refer to these two hidden state factors are `DID_IT_RAIN` and `WAS_SPRINKLER_ON`. 

#### 1. `DID_IT_RAIN`
The first factor is a binary variable representing whether or not it rained earlier today.

#### 2. `WAS_SPRINKLER_ON`

The second factor is a binary variable representing whether or not the sprinkler was on or off earlier today.

### Observations

The agent believes that these two hidden states probabilistically relate to two observation modalities, i.e. two independent 'sensory channels', which we can call `GRASS_OBSERVATION` and `WEATHER_OBSERVATION`. 

#### 1. `GRASS_OBSERVATION`
The first modality is a binary variable representing the agent's observation (e.g. via vision, for instance) of the grass being wet or being dry.

#### 2. `WEATHER_OBSERVATION`

The second modality is a ternary (3-valued) variable representing the agent's observation of the state of the weather, e.g. by looking at the sky. In this example, it can either look `clear`, `rainy`, or `cloudy`


In [ ]:
model_labels = {
            "observations": {
                "grass_observation": [
                    "wet",
                    "dry"            
                    ],
                "weather_observation": [
                    "clear",
                    "rainy",
                    "cloudy"
                ]
            },
            "states": {
                "did_it_rain": ["rained", "did_not_rain"],
                "was_sprinkler_on": ["on", "off"],
            },
        }

num_obs, _, n_states, n_factors = utils.get_model_dimensions_from_labels(model_labels)

read_from_excel = True
pre_specified_excel = True

A_stub = create_A_matrix_stub(model_labels)

### Option 1. Write the empty A matrix stub to an excel file, fill it out separately (e.g. manually in excel, and then read it back into memory). Remember, these represent the agent's generative model, not the true probabilities that relate states to observations. So you can think of these as the agent's personal/subjective 'assumptions' about how hidden states relate to observations.

In [ ]:
if read_from_excel:
    ## Option 1: fill out A matrix 'offline' (e.g. in an excel spreadsheet)

    excel_dir = 'tmp_dir'
    if not os.path.exists(excel_dir):
        os.mkdir(excel_dir)

    excel_path = os.path.join(excel_dir, 'my_a_matrix.xlsx')

    if not pre_specified_excel:
        A_stub.to_excel(excel_path)
        print(f'Go fill out the A matrix in {excel_path} and then continue running this code\n')

After you've filled out the Excel sheet separately (e.g. opening up Microsoft Excel and filling out the cells, you can read it back into memory)

In [ ]:
if read_from_excel:
    A_stub = read_A_matrix(excel_path, n_factors)

### Option 2. Fill out the A matrix using the desired probabilities. Remember, these represent the agent's generative model, not the true probabilities that relate states to observations. So you can think of these as the agent's personal/subjective 'assumptions' about how hidden states relate to observations.

In [ ]:
if not read_from_excel:
    A_stub.loc[('grass_observation','wet'),('rained', 'on')] = 1.0

    A_stub.loc[('grass_observation','wet'),('rained', 'off')] = 0.7
    A_stub.loc[('grass_observation','dry'),('rained', 'off')] = 0.3

    A_stub.loc[('grass_observation','wet'),('did_not_rain', 'on')] = 0.5
    A_stub.loc[('grass_observation','dry'),('did_not_rain', 'on')] = 0.5

    A_stub.loc[('grass_observation','dry'),('did_not_rain', 'off')] = 1.0

    A_stub.loc['weather_observation','rained'] = np.tile(np.array([0.1, 0.65, 0.25]).reshape(-1,1), (1,2)) 

    A_stub.loc[('weather_observation'),('did_not_rain')] = np.tile(np.array([0.9, 0.05, 0.05]).reshape(-1,1), (1,2)) 


### Now we can use a utility function `convert_stub_to_ndarray` to convert the human-readable A matrix into the multi-dimensional tensor form needed by `pymdp` to achieve things like inference and action selection

In [ ]:
A = utils.convert_A_stub_to_ndarray(A_stub, model_labels)

## Sample a random observation

In [ ]:
obs_idx = [np.random.randint(o_dim) for o_dim in num_obs]
# obs_idx = [0, 1] # wet and rainy

observation = utils.obj_array_zeros(num_obs)

for g, modality_name in enumerate(model_labels['observations'].keys()):
    observation[g][obs_idx[g]] = 1.0
    print('%s: %s'%(modality_name, model_labels['observations'][modality_name][obs_idx[g]]))

## Given the observation and your A matrix, perform inference to optimize a simple posterior belief about the state of the world 

In [ ]:
qs = run_vanilla_fpi(A, observation, num_obs, n_states, prior=None, num_iter=10, dF=1.0, dF_tol=0.001)

print('Belief that it rained: %.2f'%(qs[0][0]))
print('Belief that the sprinkler was on: %.2f'%(qs[1][0]))